In [1]:
import sys
sys.path.append('../lib')
import json
import pandas as pd
import matplotlib.pyplot as plt
from random_eraser import get_random_eraser
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import cv2
import json
# import symspellpy.editdistance 
from PIL import Image

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 5)

In [2]:
labels = json.load(open('../labels/labels.json'))

FileNotFoundError: [Errno 2] No such file or directory: '../labels/labels.json'

In [ ]:
y = pd.DataFrame(list(labels.values()), columns=['y'])

In [ ]:
y['length'] = y['y'].str.len()
y['comma'] = y['y'].str.split(',').str.len()

In [ ]:
y.head()

In [ ]:
y['length'].value_counts().sort_index().plot.bar()

In [ ]:
y['comma'].value_counts().sort_index().plot.bar()

In [ ]:
pd.value_counts(list(' '.join(y['y'].values))).head(50).plot.bar()

In [ ]:
''.join(sorted(list(set(list(' '.join(y['y'].values))))))

In [ ]:
# Function to distort image
def elastic_transform(image, alpha, sigma, alpha_affine, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

In [ ]:
random_eraser = get_random_eraser(p=1, s_l=0.004, s_h=0.005, r_1=0.01, r_2=1/0.01, v_l=0, v_h=1)

In [ ]:
plt.figure(figsize=(20, 10))
img = image.load_img('../data/0000_samples.png', target_size=(160, 2560), interpolation='bicubic')
img = image.img_to_array(img)/255
plt.imshow(img)

img_eraser = random_eraser(img)
for _ in range(4):
    img_eraser = random_eraser(img_eraser)


plt.figure(figsize=(20, 10))
plt.imshow(img_eraser)

In [ ]:
image_datagen_args = {
                'shear_range': 0.1,
                'zoom_range': 0.01,
                'width_shift_range': 0.001,
                'height_shift_range': 0.1,
                'rotation_range': 1,
                'horizontal_flip': False,
                'vertical_flip': False
        }
image_datagen = ImageDataGenerator(**image_datagen_args)

In [ ]:
params = image_datagen.get_random_transform(img.shape)
img_aug = image_datagen.apply_transform(img, params)
plt.figure(figsize=(20, 10))
plt.imshow(img_aug)

In [ ]:
for _ in range(5):
    plt.figure(figsize=(20, 10))
    elastic_img = elastic_transform(img, 20, 2,0.7)
    plt.imshow(elastic_img)

In [ ]:
plt.figure(figsize=(20, 10))
img = image.load_img('../test/0006_tests.png')
img = image.img_to_array(img)/255
plt.imshow(img)
plt.figure(figsize=(20, 10))
img = image.load_img('../data/1814_samples.png')
img = image.img_to_array(img)/255
plt.imshow(img)

In [ ]:
test_labels = json.load(open('../1015_Private Test/labels.json'))
predict = json.load(open('../labels/labels.json'))

In [ ]:
import editdistance

In [ ]:
test1 = 'Số 10, đường Lý Văn Lâm, Phường 1, Thành Phố Cà Mau, Cà Mau'
test2 = 'Số 0, đường L Văn Lm, Phường 1, Thành Phố C Mau, Cà Mau'

In [ ]:
editdistance.eval(test1, test2)

In [ ]:
# mean_ed = 0.0
# mean_norm_ed = 0.0
# num = len(test_labels)
# for k, v in test_labels.items():
#     edit_dist = editdistance.eval(v, predict[k])
#     mean_ed += float(edit_dist)
#     mean_norm_ed += float(edit_dist) / max(len(predict[k]), len(v))
    
# mean_norm_ed = mean_norm_ed / num
# mean_ed = mean_ed / num

# print("edit distance: {} - normalize edit distance: {} ".format(mean_ed, mean_norm_ed))


In [ ]:
img = image.load_img('../data/1814_samples.png')
img = image.img_to_array(img)/255
plt.imshow(img)

In [ ]:
test3 = cv2.imread('../data/1814_samples.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img)

In [ ]:
test4 = cv2.imread('../data/1814_samples.png')
plt.imshow(test4, cmap="gray", vmin=0, vmax=255)